# Tutorial 1: Launching a REXEE simulation

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/wehs7661/ensemble_md/c0f6d48ce3fe746e349e4a4a9610f935cca8b0b5?urlpath=lab%2Ftree%2Fdocs%2Fexamples%2Ftutorial_1%2Frun_REXEE.ipynb)

In this tutorial, we will demonstrate how one can launch a REXEE simulation for a toy system using the CLI `run_REXEE` implemented in the python package `ensemble_md`. Here, our test system is molecule composed of four interaction sites in a box of water molecules. In the next tutorials, we will demonstrate how to analyze the output of the simulation, including the calculation of the solvation free energy of the molecule. For a more comprehensive understanding, we strongly recommend reading our [documentation](https://ensemble-md.readthedocs.io/en/latest/simulations.html) on launching REXEE simulations before starting this tutorial. The tutorial can be run on binder (without installation of anything) by clicking the badge above. 

## 1. Prepare the simulation inputs

In the folder where this tutorial resides (`docs/examples/tutoria_1` in the repository), you will find all the necessary input files for this tutorial, including a GROMACS GRO file (`sys.gro`), a GROMACS TOP file (`sys.top`), a GROMACS MDP file (`expanded.mdp`), and a YAML file (`params.yaml`) that specifies input REXEE parameters.

In [1]:
!ls

expanded.mdp    params.yaml     run_REXEE.ipynb sys.gro         sys.top


As shown in the MDP file `expanded.mdp`, we have defined 9 alchemical intermediate states in total to decouple the van der Waals interactions and coulombic interactions. To decide the number of replicas and the state shift we would like to use in our REXEE simulation, we can use the CLI `explore_REXEE` to first enumerate all the possible REXEE configurations. (For more information on the CLI `explore_REXEE`, please run `explore_REXEE --help` in the terminal.)

In [2]:
!explore_REXEE -N 9

Exploration of the REXEE parameter space
[ REXEE parameters of interest ]
- N: The total number of states
- r: The number of replicas
- n: The number of states for each replica
- s: The state shift between adjacent replicas

[ Solutions ]
- Solution 1: (N, r, n, s) = (9, 2, 5, 4)
  - Replica 0: [0, 1, 2, 3, 4]
  - Replica 1: [4, 5, 6, 7, 8]

- Solution 2: (N, r, n, s) = (9, 2, 6, 3)
  - Replica 0: [0, 1, 2, 3, 4, 5]
  - Replica 1: [3, 4, 5, 6, 7, 8]

- Solution 3: (N, r, n, s) = (9, 2, 7, 2)
  - Replica 0: [0, 1, 2, 3, 4, 5, 6]
  - Replica 1: [2, 3, 4, 5, 6, 7, 8]

- Solution 4: (N, r, n, s) = (9, 2, 8, 1)
  - Replica 0: [0, 1, 2, 3, 4, 5, 6, 7]
  - Replica 1: [1, 2, 3, 4, 5, 6, 7, 8]

- Solution 5: (N, r, n, s) = (9, 3, 5, 2)
  - Replica 0: [0, 1, 2, 3, 4]
  - Replica 1: [2, 3, 4, 5, 6]
  - Replica 2: [4, 5, 6, 7, 8]

- Solution 6: (N, r, n, s) = (9, 3, 7, 1)
  - Replica 0: [0, 1, 2, 3, 4, 5, 6]
  - Replica 1: [1, 2, 3, 4, 5, 6, 7]
  - Replica 2: [2, 3, 4, 5, 6, 7, 8]

- Solution 7: (

Usually, a reasonable choice for the number of replicas would be a factor of available CPU cores on the node for running the simulation. For example, if you are runnin the simulation on a 32-core node, 4 replicas would be a good choice. To list all possible configurations that use 4 replicas for sampling 9 alchemical states, one can run the following:

In [3]:
!explore_REXEE -N 9 -r 4

Exploration of the REXEE parameter space
[ REXEE parameters of interest ]
- N: The total number of states
- r: The number of replicas
- n: The number of states for each replica
- s: The state shift between adjacent replicas

[ Solutions ]
- Solution 1: (N, r, n, s) = (9, 4, 3, 2)
  - Replica 0: [0, 1, 2]
  - Replica 1: [2, 3, 4]
  - Replica 2: [4, 5, 6]
  - Replica 3: [6, 7, 8]

- Solution 2: (N, r, n, s) = (9, 4, 6, 1)
  - Replica 0: [0, 1, 2, 3, 4, 5]
  - Replica 1: [1, 2, 3, 4, 5, 6]
  - Replica 2: [2, 3, 4, 5, 6, 7]
  - Replica 3: [3, 4, 5, 6, 7, 8]



Since a higher overlap between adjacent state sets generally facilitates 

In [4]:
!cat params.yaml

# User-defined parameters
gmx_executable: 'gmx'               
gro: sys.gro
top: sys.top
mdp: expanded.mdp             
n_sim: 4
n_iter: 5         
s: 1                         
nst_sim: 500
runtime_args: {'-nt': '1', '-ntmpi': '1'}


In our case, the system of interest is a linear model composed of 4 interaction sites, where the first and last atom have a charge of +0.2 and -0.2, respectively. In `expanded.mdp`, we define 9 alchemical states in total to decouple the van der Waals interactions and coulombic interactions. Our goal is to run an ensemble composed of 4 replicas of expanded ensemble, each of which sample 6 alchemical states, with the shift between adjacent replicas being 1 state. That is, we want replicas 0 to 3 to sample states 0 to 6, 1 to 7, 2 to 8, and 3 to 9, respectively. Each replica will be performed for 5 iterations, with the length of each iteration being 500 steps (i.e. 1 ps). We will use the Metropolis MC scheme to swap replicas, and use exponential averaging with histogram correction for combining weights of exchanging replicas. The histogram cutoff will be set as 1000 to avoid overcorrection and we will swap 2 pairs of replicas in each attempt when possible. All STDOUT will be redirected to `results.txt`. 

## 2. Run the REXEE simulation
With the input files above, we can now run the REXEE simulation using the CLI `run_REXEE` withthe following command:

In [5]:
!mpirun -np 4 run_REXEE

Current time: 17/05/2024 14:30:29
Command line: /Users/Wei-TseHsu/miniconda3/bin/run_REXEE

Important parameters of REXEE
Python version: 3.8.12 | packaged by conda-forge | (default, Jan 30 2022, 23:36:06) 
[Clang 11.1.0 ]
GROMACS executable: /usr/local/bin/gmx
GROMACS version: 2022.3-dev-20230426-4eabaf582b
ensemble_md version: 0.9.0+140.gc0f6d48.dirty
Simulation inputs: sys.gro, sys.top, expanded.mdp
Verbose log file: True
Proposal scheme: exhaustive
Whether to perform weight combination: False
Type of means for weight combination: simple
Whether to perform histogram correction: False
Histogram cutoff for weight correction: -1
Number of replicas: 4
Number of iterations: 5
Length of each replica: 1.0 ps
Frequency for checkpointing: 100 iterations
Total number of states: 9
Additionally defined swappable states: None
Additional grompp arguments: None
Additional runtime arguments: {'-nt': '1', '-ntmpi': '1'}
External modules for coordinate manipulation: None
MDP parameters differing acro